In [1]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/VRD-IU

/content/drive/MyDrive/VRD-IU


In [4]:
!cp train_data_graphs.bin train_visual_features.zip /content/

In [5]:
%cd /content/

/content


In [6]:
!unzip -q train_visual_features.zip

In [7]:
from torch.utils.data import Dataset, DataLoader
import pickle
import dgl
import os
import torch
import numpy as np
class GraphDataset(Dataset):
    def __init__(self, graph_file, feature_dir):
        super().__init__()
        self.graphs,_ = dgl.load_graphs(graph_file)

        self.feature_dir = feature_dir

    def __len__(self):
        return len(self.graphs)

    def load_feat(self,nodes):
        tensors = []
        for idx,node in enumerate(nodes):
            try:
                tensors.append(torch.load(f"{self.feature_dir}/{node}.pt",map_location=torch.device("cpu"),weights_only=False).unsqueeze(0))
            except:
                tensors.append(torch.zeros((1,1536)))
        return torch.cat(tensors,dim=0)

    def __getitem__(self, index):
        g = self.graphs[index]
        nodes = g.ndata['obj_id']
        feats = self.load_feat(nodes)
        return g, feats


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [8]:
import torch
import torch.nn as nn
import dgl
import torch.nn.functional as F
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [9]:
dataset = GraphDataset('train_data_graphs.bin','train_visual_features')

In [10]:
from torch.utils.data import random_split
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
train_dataset, val_dataset = random_split(dataset,[0.9,0.1])

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=1, collate_fn=lambda batch: batch[0])
val_dataloader = DataLoader(val_dataset, batch_size=1, collate_fn=lambda batch: batch[0])

In [19]:
model = MLPPredictor(1536)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
best_auc = 0.0
for epoch in tqdm(range(100)):
    total_loss = 0.0
    model.train()
    for g, feats in train_dataloader:
        scores = model(g.to(device),feats.to(device))
        labels = g.edata['labels']
        loss = F.binary_cross_entropy_with_logits(scores, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        g.to(torch.device('cpu'))
        feats.to(torch.device('cpu'))
        torch.cuda.empty_cache()
    total_loss/=len(train_dataloader)
    print(f'Epoch: {epoch} Train Loss: {total_loss}')
    model.eval()
    total_loss = 0.0
    auc = 0.0
    with torch.no_grad():
        for g, feats in val_dataloader:
            scores = model(g.to(device),feats.to(device))
            labels = g.edata['labels']
            loss = F.binary_cross_entropy_with_logits(scores, labels.to(device))
            total_loss+=loss.item()
            auc+= roc_auc_score(labels.detach().cpu().numpy(), scores.detach().cpu().numpy())
        total_loss/=len(val_dataloader)
        auc/=len(val_dataloader)
    print(f'Epoch: {epoch} Val Loss: {total_loss} AUC: {auc}')
    if auc > best_auc:
        best_auc = auc
        torch.save(model.state_dict(),"predictor.pth")



  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0 Train Loss: 0.08436080297458823


  1%|          | 1/100 [00:23<38:48, 23.52s/it]

Epoch: 0 Val Loss: 0.05941581173816271 AUC: 0.8803037150593642
Epoch: 1 Train Loss: 0.06327812466225778


  2%|▏         | 2/100 [00:47<38:23, 23.50s/it]

...

Epoch: 97 Val Loss: 0.06469791421764776 AUC: 0.8949546384677456
Epoch: 98 Train Loss: 0.04384132066164154


 99%|█████████▉| 99/100 [38:10<00:23, 23.14s/it]

Epoch: 98 Val Loss: 0.06514908059647209 AUC: 0.8930278870925165
Epoch: 99 Train Loss: 0.044661336168214306


100%|██████████| 100/100 [38:33<00:00, 23.14s/it]

Epoch: 99 Val Loss: 0.06556548812148864 AUC: 0.891405585958479


In [21]:
best_auc

0.900139497817671